In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
import nltk.stem.porter as porter
from collections import Counter

In [2]:
data = pd.read_csv('tf-df.csv')

In [3]:
doc_headers = [column for column in data.columns if 'Document' in column]

In [98]:
def ntc_normalize(tf, df):
    vecs = tf.copy()
    norm_doc_freq = np.log(len(vecs.columns)/df)
    # tf.df
    vecs = vecs.multiply(norm_doc_freq, axis='rows')
    # Cosine normalization:
    sum_sq = np.sum(vecs**2, axis=0)
    vecs = vecs/np.sqrt(sum_sq)
    return vecs

In [99]:
def nnc_normalize(tf, df):
    vecs = tf.copy()
    norm_doc_freq = 1
    # tf.df
    #vecs = vecs.multiply(norm_doc_freq, axis='rows')
    # Cosine normalization:
    sum_sq = np.sum(vecs**2, axis=0)
    vecs = vecs/np.sqrt(sum_sq)
    return vecs

In [6]:
def get_stemmed_tokenized_query(query):
    stemmer = porter.PorterStemmer(mode='ORIGINAL_ALGORITHM')
    #get tokenized words from query
    words = word_tokenize(query)
    #get stemmed words from query
    words = list(map(stemmer.stem, words))
    return words

In [7]:
# word_series is the series of words in our data, i.e data['word']
def get_query_tf(query, word_series):
    tf = word_series.copy()
    words = get_stemmed_tokenized_query(query)
    #count the occurences
    freq = Counter(words)
    #set the value as the count if it exists, else 0
    tf = tf.transform(lambda word: freq[word] if word in freq else 0)
    return tf

In [8]:
query_tf = get_query_tf("I enjoy the fall content", data['word'])

In [257]:
norm_query = nnc_normalize(query_tf, data['df'])

In [10]:
norm_vecs = ntc_normalize(data[doc_headers], data['df'])

In [265]:
norm_vecs.multiply(norm_query, axis=0).sum(axis=0).idxmax()

'Document 6'

In [268]:
doc_headers.index('Document 6')

5

In [269]:
dc = doc_headers.copy()

In [11]:
def get_similarity_scores(norm_docs, norm_query):
    cosine_prod = norm_vecs.multiply(norm_query, axis=0)
    return cosine_prod.sum(axis=0)

In [12]:
scores = get_similarity_scores(norm_vecs, norm_query)

In [13]:
#Index of maximum, which is our most similar document
scores.idxmax()

'Document 15'

In [272]:
scores['Document 1']

0.00011359120365123902

In [274]:
len(titles_previews)

31

In [276]:
titles_previews.query('Document in '+ str(doc_headers[:3]))

,Document,title,preview,url
0,Document 1,freeman moore - smu spring 2018,"Spring 2018 Freeman L. Moore , PhD email : fmo...",https://s2.smu.edu/~fmoore/
1,Document 2,smu cse 5337/7337 spring 2018 schedule,SMU CSE 5337/7337 2018 Preliminary Schedule Th...,https://s2.smu.edu/~fmoore/schedule.htm
2,Document 3,smu cse 5/7337 spring 2018 textfiles,Textfiles for clustering golf 1 golf 2 golf 3 ...,https://s2.smu.edu/~fmoore/textfiles/index.html


In [15]:
titles_previews = pd.read_csv('title_preview.csv')

In [16]:
titles_previews = titles_previews.fillna("")

In [17]:
titles_previews['title'] = titles_previews['title'].apply(lambda word: word.lower())

In [18]:
def get_score(string, words):
    intersection = [word for word in words if word in string]
    return 0.25 if len(intersection)>0 else 0

In [19]:
def get_score_by_title(scores, titles_previews, query_words):
    modif = scores.copy()
    titles = titles_previews['title'].copy()
    titles.index = modif.index
    titles = titles.transform(lambda string: get_score(string, words))
    return titles+modif

In [20]:
query_words = get_stemmed_tokenized_query("I enjoy the fall content")

In [21]:
query_words

['i', 'enjoi', 'the', 'fall', 'content']

In [22]:
scores[scores.values > 0]

Document 1     0.000114
Document 2     0.000010
Document 5     0.000036
Document 9     0.000022
Document 10    0.000062
Document 12    0.000040
Document 13    0.000056
Document 14    0.000045
Document 15    0.000116
Document 16    0.000086
Document 17    0.000028
Document 18    0.000086
Document 19    0.000065
Document 21    0.000102
Document 22    0.000096
Document 24    0.000072
Document 26    0.000096
Document 27    0.000045
Document 31    0.000011
dtype: float64

In [23]:
tp = titles_previews.copy()

In [70]:
import scorer

In [28]:
query_tf = scorer.get_query_tf("buildingone buildingtwo buildingthree", data['word'])

In [31]:
norm_query = scorer.nnc_normalize(query_tf, data['df'])

In [34]:
norm_query[norm_query!=0]

233    0.166667
234    0.166667
235    0.166667
dtype: float64

In [36]:
doc_tf = data[doc_headers]

In [38]:
norm_doc = scorer.nnc_normalize(doc_tf, data['df'])

In [117]:
import scorer

In [ ]:
scorer.

In [119]:
scorer.data[doc_headers]

,Document 1,Document 2,Document 3,Document 4,Document 5,Document 6,Document 7,Document 8,Document 9,Document 10,...,Document 22,Document 23,Document 24,Document 25,Document 26,Document 27,Document 28,Document 29,Document 30,Document 31
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1,0,0,2,0,0,0,2,4,...,1,0,1,0,1,1,0,0,0,0
2,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
4,1,2,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
7,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
8,4,2,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
9,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [118]:
scorer.norm_doc_vec

,Document 1,Document 2,Document 3,Document 4,Document 5,Document 6,Document 7,Document 8,Document 9,Document 10,...,Document 22,Document 23,Document 24,Document 25,Document 26,Document 27,Document 28,Document 29,Document 30,Document 31
0,0.000484,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1,0.016457,0.007375,0.000000,0.000000,0.008038,0.0,0.0,0.0,0.004996,0.006827,...,0.012292,0.000000,0.017103,0.0,0.028765,0.012889,0.000000,0.000000,0.0,0.000000
2,0.001452,0.001302,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.003055
3,0.000968,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000588,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,0.001936,0.003471,0.002527,0.002269,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
5,0.000968,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
6,0.001452,0.000000,0.000000,0.000000,0.000709,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.002211,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
7,0.001452,0.001302,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.012766,0.0,0.000000
8,0.011617,0.005206,0.003790,0.003403,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.021978,0.000000,0.0,0.000000
9,0.000968,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [39]:
norm_doc

,Document 1,Document 2,Document 3,Document 4,Document 5,Document 6,Document 7,Document 8,Document 9,Document 10,...,Document 22,Document 23,Document 24,Document 25,Document 26,Document 27,Document 28,Document 29,Document 30,Document 31
0,0.000484,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1,0.016457,0.007375,0.000000,0.000000,0.008038,0.0,0.0,0.0,0.004996,0.006827,...,0.012292,0.000000,0.017103,0.0,0.028765,0.012889,0.000000,0.000000,0.0,0.000000
2,0.001452,0.001302,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.003055
3,0.000968,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000588,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,0.001936,0.003471,0.002527,0.002269,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
5,0.000968,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
6,0.001452,0.000000,0.000000,0.000000,0.000709,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.002211,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
7,0.001452,0.001302,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.012766,0.0,0.000000
8,0.011617,0.005206,0.003790,0.003403,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.021978,0.000000,0.0,0.000000
9,0.000968,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [52]:
demo = data.copy()

In [53]:
demo = demo.loc[:8, doc_headers[:3]]

In [55]:
demo['Document 1'] = np.array([0, 1, 0, 0, 0, 1, 1, 1, 1])
demo['Document 2'] = np.array([1, 1, 1, 0, 1, 1, 1, 0, 0])
demo['Document 3'] = np.array([0, 0, 0, 1, 1, 2, 0, 0, 1])

In [60]:
demo['df'] = np.sum(demo, axis=1)

In [61]:
demo

,Document 1,Document 2,Document 3,df
0,0,1,0,1
1,1,1,0,2
2,0,1,0,1
3,0,0,1,1
4,0,1,1,2
5,1,1,2,4
6,1,1,0,2
7,1,0,0,1
8,1,0,1,2


In [67]:
demo['word'] = ['is', 'made', 'of', 'or', 'steel', 'table', 'the', 'they', 'wood']

In [68]:
demo

,Document 1,Document 2,Document 3,df,word
0,0,1,0,1,is
1,1,1,0,2,made
2,0,1,0,1,of
3,0,0,1,1,or
4,0,1,1,2,steel
5,1,1,2,4,table
6,1,1,0,2,the
7,1,0,0,1,they
8,1,0,1,2,wood


In [65]:
query = "the wood table"

In [73]:
def get_query_tf(query, word_series):
    tf = word_series.copy()
    words = word_tokenize(query)
    
    # count the occurrences
    freq = Counter(words)
    # set the value as the count if it exists, else 0
    tf = tf.transform(lambda word: freq[word] if word in freq else 0)
    return tf

In [75]:
q = get_query_tf("the wood table", demo['word'])

In [103]:
vq = nnc_normalize(q, demo['df'])

In [102]:
dd = nnc_normalize(demo[doc_headers[:3]], demo['df'])

In [104]:
scorer.get_similarity_scores(dd, vq)

Document 1    0.774597
Document 2    0.471405
Document 3    0.654654
dtype: float64

In [116]:
np.arange(0, 25, 5)

array([ 0,  5, 10, 15, 20])

In [121]:
data2 = data.copy()

In [123]:
import scorer

In [124]:
scorer.lnc_normalize(data2[doc_headers], data2['df'])

,Document 1,Document 2,Document 3,Document 4,Document 5,Document 6,Document 7,Document 8,Document 9,Document 10,...,Document 22,Document 23,Document 24,Document 25,Document 26,Document 27,Document 28,Document 29,Document 30,Document 31
0,0.000773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.022250,0.013758,NaN,NaN,0.011176,NaN,NaN,NaN,0.010060,0.009959,...,0.020227,NaN,0.018495,NaN,0.029558,0.015097,NaN,NaN,NaN,NaN
2,0.002319,0.002428,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.006042
3,0.001546,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001183,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.003092,0.005481,0.007918,0.006011,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.001546,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.002319,NaN,NaN,NaN,0.001165,NaN,NaN,NaN,NaN,NaN,...,NaN,0.002919,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0.002319,0.002428,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.013085,NaN,NaN
8,0.011068,0.008222,0.011876,0.009016,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.028480,NaN,NaN,NaN
9,0.001546,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [254]:
def ltc_normalize(tf, df):
    vecs = tf.copy()
    # log normalization
    vecs = vecs[vecs!=0]
    vecs = np.log(vecs) + 1
    vecs = vecs.fillna(1)
    # df normalization
    norm_doc_freq = np.log(len(doc_headers) / df)
    # tf.df
    vecs = vecs.multiply(norm_doc_freq, axis='rows')
    # Cosine normalization:
    sum_sq = np.sum(vecs**2, axis=0)
    vecs = vecs/np.sqrt(sum_sq)
    return vecs

In [255]:
ltc_normalize(data2[doc_headers], data2['df'])

,Document 1,Document 2,Document 3,Document 4,Document 5,Document 6,Document 7,Document 8,Document 9,Document 10,...,Document 22,Document 23,Document 24,Document 25,Document 26,Document 27,Document 28,Document 29,Document 30,Document 31
0,0.031497,0.030219,0.031682,0.031664,0.031395,0.031797,0.031797,0.031778,0.031143,0.031366,...,0.031729,0.031402,0.031697,0.031621,0.031734,0.031651,0.031679,0.031712,0.030676,0.030756
1,0.009330,0.005287,0.005543,0.005540,0.009300,0.005563,0.005563,0.005559,0.009225,0.013095,...,0.005551,0.005494,0.005545,0.005532,0.005552,0.005537,0.005542,0.005548,0.005367,0.005381
2,0.021420,0.020551,0.021546,0.021534,0.021351,0.021624,0.021624,0.021611,0.021180,0.021331,...,0.021578,0.021356,0.021557,0.021505,0.021582,0.021525,0.021544,0.021567,0.020862,0.020916
3,0.025139,0.024119,0.025287,0.025273,0.025058,0.025379,0.025379,0.025363,0.042087,0.025035,...,0.025324,0.025063,0.025299,0.025238,0.025329,0.025262,0.025284,0.025311,0.024484,0.024548
4,0.018782,0.030510,0.018892,0.018882,0.018721,0.018961,0.018961,0.018949,0.018571,0.018704,...,0.018920,0.018725,0.018901,0.018856,0.018923,0.018873,0.018890,0.018910,0.018292,0.018340
5,0.025139,0.024119,0.025287,0.025273,0.025058,0.025379,0.025379,0.025363,0.024857,0.025035,...,0.025324,0.025063,0.025299,0.025238,0.025329,0.025262,0.025284,0.025311,0.024484,0.024548
6,0.021420,0.020551,0.021546,0.021534,0.021351,0.021624,0.021624,0.021611,0.021180,0.021331,...,0.021578,0.021356,0.021557,0.021505,0.021582,0.021525,0.021544,0.021567,0.020862,0.020916
7,0.021420,0.020551,0.021546,0.021534,0.021351,0.021624,0.021624,0.021611,0.021180,0.021331,...,0.021578,0.021356,0.021557,0.021505,0.021582,0.021525,0.021544,0.021567,0.020862,0.020916
8,0.035944,0.024469,0.015151,0.015143,0.015014,0.015206,0.015206,0.015197,0.014894,0.015000,...,0.015174,0.015017,0.015158,0.015122,0.015176,0.015136,0.015150,0.015166,0.014670,0.014708
9,0.025139,0.024119,0.025287,0.025273,0.025058,0.025379,0.025379,0.025363,0.024857,0.025035,...,0.025324,0.025063,0.025299,0.025238,0.025329,0.025262,0.025284,0.025311,0.024484,0.024548


In [188]:
non_zero.fillna(1)

,Document 1,Document 2,Document 3,Document 4,Document 5,Document 6,Document 7,Document 8,Document 9,Document 10,...,Document 22,Document 23,Document 24,Document 25,Document 26,Document 27,Document 28,Document 29,Document 30,Document 31
0,1.000000,1.000000,1.0,1.0,1.000000,1.0,1.0,1.0,1.000000,1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.000000
1,1.693147,1.000000,1.0,1.0,1.693147,1.0,1.0,1.0,1.693147,2.386294,...,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.000000
2,1.000000,1.000000,1.0,1.0,1.000000,1.0,1.0,1.0,1.000000,1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.000000
3,1.000000,1.000000,1.0,1.0,1.000000,1.0,1.0,1.0,1.693147,1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.000000
4,1.000000,1.693147,1.0,1.0,1.000000,1.0,1.0,1.0,1.000000,1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.000000
5,1.000000,1.000000,1.0,1.0,1.000000,1.0,1.0,1.0,1.000000,1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.000000
6,1.000000,1.000000,1.0,1.0,1.000000,1.0,1.0,1.0,1.000000,1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.000000
7,1.000000,1.000000,1.0,1.0,1.000000,1.0,1.0,1.0,1.000000,1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.000000
8,2.386294,1.693147,1.0,1.0,1.000000,1.0,1.0,1.0,1.000000,1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.000000
9,1.000000,1.000000,1.0,1.0,1.000000,1.0,1.0,1.0,1.000000,1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.000000


In [256]:
scores

Document 1     0.000114
Document 2     0.000010
Document 3     0.000000
Document 4     0.000000
Document 5     0.000036
Document 6     0.000000
Document 7     0.000000
Document 8     0.000000
Document 9     0.000022
Document 10    0.000062
Document 11    0.000000
Document 12    0.000040
Document 13    0.000056
Document 14    0.000045
Document 15    0.000116
Document 16    0.000086
Document 17    0.000028
Document 18    0.000086
Document 19    0.000065
Document 20    0.000000
Document 21    0.000102
Document 22    0.000096
Document 23    0.000000
Document 24    0.000072
Document 25    0.000000
Document 26    0.000096
Document 27    0.000045
Document 28    0.000000
Document 29    0.000000
Document 30    0.000000
Document 31    0.000011
dtype: float64

In [155]:
def log_norm(value):
    new_val = 0 if value==0 else np.log(value)
    return new_val+1

In [146]:
vecs.transform(lambda value: 0 if 0 else np.log(value)+1)

/Users/adi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


,Document 1,Document 2,Document 3,Document 4,Document 5,Document 6,Document 7,Document 8,Document 9,Document 10,...,Document 22,Document 23,Document 24,Document 25,Document 26,Document 27,Document 28,Document 29,Document 30,Document 31
0,1.000000,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,...,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
1,1.693147,1.000000,-inf,-inf,1.693147,-inf,-inf,-inf,1.693147,2.386294,...,1.000000,-inf,1.000000,-inf,1.000000,1.000000,-inf,-inf,-inf,-inf
2,1.000000,1.000000,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,...,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,1.000000
3,1.000000,-inf,-inf,-inf,-inf,-inf,-inf,-inf,1.693147,-inf,...,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
4,1.000000,1.693147,1.000000,1.000000,-inf,-inf,-inf,-inf,-inf,-inf,...,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
5,1.000000,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,...,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
6,1.000000,-inf,-inf,-inf,1.000000,-inf,-inf,-inf,-inf,-inf,...,-inf,1.000000,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
7,1.000000,1.000000,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,...,-inf,-inf,-inf,-inf,-inf,-inf,-inf,1.000000,-inf,-inf
8,2.386294,1.693147,1.000000,1.000000,-inf,-inf,-inf,-inf,-inf,-inf,...,-inf,-inf,-inf,-inf,-inf,-inf,1.000000,-inf,-inf,-inf
9,1.000000,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,...,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
